In [ ]:
####################### Comparision of Conversion of Bidding Methods with AB Test   #######################

##########  Business Problem :

# Evaluating which method has higher rate of purchase among maximum bidding and average bidding with AB test


##########  Dataset:


#  Control Group: Maximum Bidding
#  Test Group :  Average Bidding


# impression: Advertisement view count
# Click: Click counts of the viewed advertisement
# Purchase: Number of products purchased after ads clicked
# Earning: Earnings after purchased products

In [ ]:
##########  TASK 1 :  Data Preparation and Analysis

import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import shapiro, levene, ttest_ind


pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [6]:
# New variables for test group and control group

dataframe_control = pd.read_excel(r"C:\Users\burcu\OneDrive\Masaüstü\DS Miiul\2.Measurement Problems\ALIŞTIRMALAR\1.Case 1 AB Testiyle Bidding Yötemlerinni Karşılaştırılması\tanıtım ve datasetler\ab_testing.xlsx",
                                  sheet_name="Control Group")
dataframe_test = pd.read_excel(r"C:\Users\burcu\OneDrive\Masaüstü\DS Miiul\2.Measurement Problems\ALIŞTIRMALAR\1.Case 1 AB Testiyle Bidding Yötemlerinni Karşılaştırılması\tanıtım ve datasetler\ab_testing.xlsx",
                                   sheet_name="Test Group")

df_control = dataframe_control.copy()
df_test = dataframe_test.copy()

In [7]:
# 2. Analysis of Test and Control Groups

def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head())
    print("##################### Tail #####################")
    print(dataframe.tail())
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.quantile([0, 0.05, 0.50, 0.95, 0.99, 1]).T)

check_df(df_control)
check_df(df_test)

##################### Shape #####################
(40, 4)
##################### Types #####################
Impression    float64
Click         float64
Purchase      float64
Earning       float64
dtype: object
##################### Head #####################
    Impression      Click  Purchase    Earning
0  82529.45927 6090.07732 665.21125 2311.27714
1  98050.45193 3382.86179 315.08489 1742.80686
2  82696.02355 4167.96575 458.08374 1797.82745
3 109914.40040 4910.88224 487.09077 1696.22918
4 108457.76263 5987.65581 441.03405 1543.72018
##################### Tail #####################
     Impression      Click  Purchase    Earning
35 132064.21900 3747.15754 551.07241 2256.97559
36  86409.94180 4608.25621 345.04603 1781.35769
37 123678.93423 3649.07379 476.16813 2187.72122
38 101997.49410 4736.35337 474.61354 2254.56383
39 121085.88122 4285.17861 590.40602 1289.30895
##################### NA #####################
Impression    0
Click         0
Purchase      0
Earning       0
dtype: int6

In [9]:
# 3. Combine control and test group data

df_control["group"] = "control"
df_test["group"] = "test"

df = pd.concat([df_control,df_test], axis=0,ignore_index=False)
df.head()

,Impression,Click,Purchase,Earning,group
0,82529.45927,6090.07732,665.21125,2311.27714,control
1,98050.45193,3382.86179,315.08489,1742.80686,control
2,82696.02355,4167.96575,458.08374,1797.82745,control
3,109914.40040,4910.88224,487.09077,1696.22918,control
4,108457.76263,5987.65581,441.03405,1543.72018,control


In [10]:
##########  TASK 2 : Defining the A/B Test Hypothesis


# H0 : M1 = M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında fark yoktur.)
# H1 : M1!= M2 (Kontrol grubu ve test grubu satın alma ortalamaları arasında fark vardır.)


# Analysis of the purchase  averages for the control and test groupdf.groupby("group").agg({"Purchase": "mean"})

df.groupby("group").agg({"Purchase": "mean"})

,Purchase
group,
control,550.89406
test,582.10610


In [11]:
##########  TASK 3 : Hypothesis Testing

# 1: Normality assumption and variance  are checked before hypotesis testing

# Testing ormality assumption according to purchase variable for test and control groups

# Normality Assumption :
# H0: Normal distribution assumption provided
# H1: Normal distribution assumption not provided
# p < 0.05 H0 reject
# p > 0.05 H0 fail to reject


test_stat, pvalue = shapiro(df.loc[df["group"] == "control", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


In [ ]:
# p-value=0.5891
# HO fail to reject. Control group is normally distributed

In [12]:
#  Homogeneity of Variance
# H0: Variances are homogeneous
# H1: Variances are not homogeneous
# p < 0.05 H0 reject
# p > 0.05 H0 fail to reject

# Testing homogeneity of variance of test and control groups according to purchase variable 

test_stat, pvalue = levene(df.loc[df["group"] == "control", "Purchase"],
                           df.loc[df["group"] == "test", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


In [ ]:
# p-value=0.1083
# HO fail to reject.
#Variances are homogeneous in both groups

In [14]:
# 2:

# Independent two-sample t-test (parametric test) is applied.
# H0: M1 = M2 (There is no statistical difference between the purchase of conrol group and the test group)
# H1: M1 != M2 (There is statistical difference between the purchase of control group and the test group)
# p<0.05 HO reject , p>0.05 HO fail to reject

test_stat, pvalue = ttest_ind(df.loc[df["group"] == "control", "Purchase"],
                              df.loc[df["group"] == "test", "Purchase"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


In [ ]:
# p-value=0.3493
# HO fail to reject. 

# There is no statistical difference between the purchase of control group and the test group
